In [7]:
def setupSpark():
  # Spark needs to run with Java 8 ... 
  !pip install -q findspark
  !apt-get install openjdk-8-jdk-headless > /dev/null
  !echo 2 | update-alternatives --config java > /dev/null
  !java -version
  import os, findspark
  os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
  # !echo JAVA_HOME=$JAVA_HOME
  !pip install -q pyspark
  findspark.init(spark_home='/usr/local/lib/python3.7/dist-packages/pyspark')
  !pyspark --version

setupSpark()

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
     |████████████████████████████████| 212.4 MB 83 kB/s 
     |████████████████████████████████| 198 kB 73.2 MB/s 
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_292
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [8]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
    
spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

In [10]:
smsData = sc.textFile("/content/spam.csv",2)
smsData.cache()
smsData.collect()

['v1,v2,,,',
 'ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",,,',
 'ham,Ok lar... Joking wif u oni...,,,',
 "spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,,,",
 'ham,U dun say so early hor... U c already then say...,,,',
 'ham,"Nah I don\'t think he goes to usf, he lives around here though",,,',
 'spam,"FreeMsg Hey there darling it\'s been 3 week\'s now and no word back! I\'d like some fun you up for it still? Tb ok! XxX std chgs to send, �1.50 to rcv",,,',
 'ham,Even my brother is not like to speak with me. They treat me like aids patent.,,,',
 "ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,,,",
 'spam,WINNER!! As a valued network customer you have been selected to receivea �900 prize

In [15]:
def TransformToVector(inputStr):
    attList=inputStr.split(",")
    smsType= 0.0 if attList[0] == "ham" else 1.0
    return [smsType, attList[1]]

smsXformed=smsData.map(TransformToVector)

smsDf= spark.createDataFrame(smsXformed,
                          ["label","message"])
smsDf.cache()
smsDf.select("label","message").show()

+-----+--------------------+
|label|             message|
+-----+--------------------+
|  1.0|                  v2|
|  0.0|"Go until jurong ...|
|  0.0|Ok lar... Joking ...|
|  1.0|Free entry in 2 a...|
|  0.0|U dun say so earl...|
|  0.0|"Nah I don't thin...|
|  1.0|"FreeMsg Hey ther...|
|  0.0|Even my brother i...|
|  0.0|As per your reque...|
|  1.0|WINNER!! As a val...|
|  1.0|Had your mobile 1...|
|  0.0|"I'm gonna be hom...|
|  1.0|"SIX chances to w...|
|  1.0|"URGENT! You have...|
|  0.0|I've been searchi...|
|  0.0|I HAVE A DATE ON ...|
|  1.0|"XXXMobileMovieCl...|
|  0.0|Oh k...i'm watchi...|
|  0.0|Eh u remember how...|
|  0.0|Fine if that��s t...|
+-----+--------------------+
only showing top 20 rows



In [16]:
(trainingData, testData) = smsDf.randomSplit([0.9, 0.1])
trainingData.count()
testData.count()
testData.collect()

[Row(label=0.0, message='" said kiss'),
 Row(label=0.0, message='"Aight'),
 Row(label=0.0, message='"Aight fuck it'),
 Row(label=0.0, message='"Al he does is moan at me if n e thin goes wrong its my fault&al de arguments r my fault&fed up of him of himso y bother? Hav 2go'),
 Row(label=0.0, message='"Alright omw'),
 Row(label=0.0, message='"As usual..iam fine'),
 Row(label=0.0, message='"Awesome'),
 Row(label=0.0, message='"Babe'),
 Row(label=0.0, message='"Babe ! How goes that day ? What are you doing ? Where are you ? I sip my cappuccino and think of you'),
 Row(label=0.0, message='"Dear Hero'),
 Row(label=0.0, message='"Go until jurong point'),
 Row(label=0.0, message='"Goodmorning'),
 Row(label=0.0, message='"Greetings me'),
 Row(label=0.0, message='"Haha awesome'),
 Row(label=0.0, message='"He has lots of used ones babe'),
 Row(label=0.0, message='"Hey sweet'),
 Row(label=0.0, message='"Hiya '),
 Row(label=0.0, message='"I can\'t'),
 Row(label=0.0, message='"I can\'t keep going th

In [17]:
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.feature import IDF
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
tokenizer = Tokenizer(inputCol="message", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), \
        outputCol="tempfeatures")
idf=IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")
nbClassifier=NaiveBayes()

pipeline = Pipeline(stages=[tokenizer, hashingTF, \
                idf, nbClassifier])

In [19]:
nbModel=pipeline.fit(trainingData)

In [20]:
prediction=nbModel.transform(testData)

In [21]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", \
                    labelCol="label",metricName="accuracy")
evaluator.evaluate(prediction)


0.9619565217391305

In [22]:
prediction.groupBy("label","prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   68|
|  0.0|       1.0|   10|
|  1.0|       0.0|   11|
|  0.0|       0.0|  463|
+-----+----------+-----+

